<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Update BDD Interactions

**Tags:** #googlesheets #gsheet #data #naas_drivers #operations #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook allows to send data to Google Sheets to a Google Sheets spreadsheet.

## Input

### Import libraries

In [ ]:
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date
import naas_data_product

### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `file_reactions`: Name of the file with reactions to be retrieved.
- `file_comments`: Name of the file with comments to be retrieved.

**Outputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `append`: If False, data will be canceled and replaced.

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())
file_reactions = "linkedin_post_reactions"
file_comments = "linkedin_post_comments"

# Outputs
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name = "INTERACTIONS"
append = False
file_name = "linkedin_interactions"

## Model

### Get data from Google Sheets spreadsheet

In [ ]:
df_gsheet = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
print("Rows:", len(df_gsheet))
df_gsheet.head(1)

### Get reactions

In [ ]:
df_reactions = pload(input_dir, file_reactions)    
print('👍 Total Reactions:', len(df_reactions))
df_reactions.head(1)

### Get comments

In [ ]:
df_comments = pload(input_dir, file_comments)    
print('🗨️ Total Comments:', len(df_comments))
df_comments.head(1)

### Cleaning data

In [ ]:
def create_interactions_dataset(
    df_gsheet,
    df_reactions,
    df_comments,
):
    # Df reactions
    data_reaction = {
        "ENTITY": df_reactions["ENTITY"],
        "SCENARIO": df_reactions["SCENARIO"],
        "PLATFORM": "LinkedIn",
        "FIRSTNAME": df_reactions["FIRSTNAME"],
        "LASTNAME": df_reactions["LASTNAME"],
        "FULLNAME": df_reactions["FULLNAME"],
        "OCCUPATION": df_reactions["OCCUPATION"],
        "INTERACTION": "POST_REACTION",
        "INTERACTION_CONTENT": df_reactions["REACTION_TYPE"],
        "INTERACTION_SCORE": 1,
        "PROFILE_URL": df_reactions["PROFILE_URL"],
        "CONTENT_TITLE": df_reactions["TITLE"],
        "CONTENT_URL": df_reactions["POST_URL"],
        "PUBLISHED_DATE": df_reactions["PUBLISHED_DATE"],
        "DATE_EXTRACT": df_reactions["DATE_EXTRACT"],
    }
    df1 = pd.DataFrame(data_reaction)
    
    # Df comments
    data_comment = {
        "ENTITY": df_comments["ENTITY"],
        "SCENARIO": df_comments["SCENARIO"],
        "PLATFORM": "LinkedIn",
        "FIRSTNAME": df_comments["FIRSTNAME"],
        "LASTNAME": df_comments["LASTNAME"],
        "FULLNAME": df_comments["FULLNAME"],
        "OCCUPATION": df_comments["OCCUPATION"],
        "INTERACTION": "POST_COMMENT",
        "INTERACTION_CONTENT": df_comments["TEXT"],
        "INTERACTION_SCORE": 3,
        "PROFILE_URL": df_comments["PROFILE_URL"],
        "CONTENT_TITLE": df_comments["TITLE"],
        "CONTENT_URL": df_comments["CONTENT_URL"],
        "PUBLISHED_DATE": df_comments["PUBLISHED_DATE"],
        "DATE_EXTRACT": df_comments["DATE_EXTRACT"],
    }
    df2 = pd.DataFrame(data_comment)
    
    # Concat df
    df = pd.concat([df1, df2]).reset_index(drop=True)
    
    # Exclude Entity from Full name
    entity = df.loc[0 , "ENTITY"]
    df = df[df["FULLNAME"] != entity]
    
    # Drop duplicates
    df = pd.concat([df, df_gsheet]).drop_duplicates(["ENTITY", "SCENARIO", "PROFILE_URL", "INTERACTION_CONTENT", "CONTENT_URL"]).reset_index(drop=True)
    return df.sort_values(by=["PUBLISHED_DATE", "FULLNAME"], ascending=[False, True]).reset_index(drop=True)

df_interactions = create_interactions_dataset(
    df_gsheet,
    df_reactions,
    df_comments,
)
print('🗂️ Interactions:', len(df_interactions))
df_interactions.head(5)

## Output

### Save data

In [ ]:
pdump(input_dir, df_interactions, file_name)

### Send data to Google Sheets spreadsheet

In [ ]:
gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name, data=df_interactions, append=append)